In [3]:
# Load files
import csv
def open_file(filename):
    with open(filename) as f:
        reader = csv.reader(f)
        header = next(reader)
        label = []
        feature = []
        for i in reader:
            label.append(i[0])
            feature.append(i[2])
        return feature, label

In [4]:
# glove
glove_train_feature, glove_train_label = open_file('D:/comp90049/project3/data/train_glove300.csv')
glove_dev_feature, glove_dev_label = open_file('D:/comp90049/project3/data/dev_glove300.csv')
glove_test_feature, glove_test_label = open_file('D:/comp90049/project3/data/test_glove300.csv')
# tfidf
tfidf_train_feature, tfidf_train_label = open_file('D:/comp90049/project3/data/train_tfidf.csv')
tfidf_dev_feature, tfidf_dev_label = open_file('D:/comp90049/project3/data/dev_tfidf.csv')
tfidf_test_feature, tfidf_test_label = open_file('D:/comp90049/project3/data/test_tfidf.csv')
# node2vec
node2vec_train_feature, node2vec_train_label = open_file('D:/comp90049/project3/data/train_node2vec300.csv')
node2vec_dev_feature, node2vec_dev_label = open_file('D:/comp90049/project3/data/dev_node2vec300.csv')
node2vec_test_feature, node2vec_test_label = open_file('D:/comp90049/project3/data/test_node2vec300.csv')

In [15]:
import numpy as np
def standard_label(label):
    dict = {"MIDWEST" : 0, "NORTHEAST" : 1, "SOUTH" : 2, "WEST" : 3}
    standard_label = []
    for l in label:
        standard_label.append(dict[l])
    return np.array(standard_label)

In [16]:
standard_glove_train_label = standard_label(glove_train_label)
standard_glove_dev_label = standard_label(glove_dev_label)
standard_node2vec_train_label = standard_label(node2vec_train_label)
standard_node2vec_dev_label = standard_label(node2vec_dev_label)

In [17]:
import ast

def standard_glove_feature(feature):
    pos = 0
    standard_glove_feature = []
    for pos in range(len(feature)):
        cur = list(map(float, feature[pos].split()))
        standard_glove_feature.append(cur)
    return np.array(standard_glove_feature)

def standard_tfidf_feature(feature):
    standard_tfidf_feature = []
    for pos in range(len(feature)):
        feature[pos] = ast.literal_eval(feature[pos])
    for f in feature:
        index_dict = {}
        for i in f:
            index_dict[i[0]] = i[1]
        one_tfidf = []
        for count in range(2037):
            if(count not in index_dict):
                one_tfidf.append(0)
            else:
                one_tfidf.append(index_dict[count])
        standard_tfidf_feature.append(one_tfidf)
    return np.array(standard_tfidf_feature)

def standard_node2vec_feature(feature):
    pos = 0
    standard_node2vec_feature = []
    for pos in range(len(feature)):
        ns = feature[pos][1:-1].replace('\n','')
        cur = list(map(float, ns.split()))
        standard_node2vec_feature.append(cur)
    return np.array(standard_node2vec_feature)

In [18]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
def open_file(filename):
    with open(filename) as f:
        reader = csv.reader(f)
        header = next(reader)
        label = {}
        feature = {}
        for i in reader:
            if(i[1] not in feature):
                feature[i[1]] = i[2]
                label[i[1]] = i[0]
            else:
                feature[i[1]] = feature[i[1]] + i[2]
        return feature, label
    
full_train_feature, full_train_label = open_file('D:/comp90049/project3/data/train_full.csv')
full_dev_feature, full_dev_label = open_file('D:/comp90049/project3/data/dev_full.csv')
full_test_feature, full_test_label = open_file('D:/comp90049/project3/data/test_full.csv')
def new_feature_label(ofeature, olabel):
    nfeature = []
    nlabel = []
    for user in ofeature:
        nfeature.append(ofeature[user])
        nlabel.append(olabel[user])
    return nfeature, nlabel
n_train_feature, n_train_label = new_feature_label(full_train_feature, full_train_label)
n_dev_feature, n_dev_label = new_feature_label(full_dev_feature, full_dev_label)
n_test_feature, n_test_label = new_feature_label(full_test_feature, full_test_label)

word_vectorizer = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='word',
        stop_words='english',
        min_df = 300)
word_vectorizer.fit(n_train_feature)
f_tfidf_train_feature = word_vectorizer.transform(n_train_feature).toarray()
f_tfidf_dev_feature = word_vectorizer.transform(n_dev_feature).toarray()
f_tfidf_test_feature = word_vectorizer.transform(n_test_feature).toarray()

In [19]:
print(len(f_tfidf_train_feature[0]))
print(len(f_tfidf_test_feature[0]))

315
315


In [20]:
# glove
glove_train_feature = standard_glove_feature(glove_train_feature)
glove_dev_feature = standard_glove_feature(glove_dev_feature)
glove_test_feature = standard_glove_feature(glove_test_feature)
# tf-idf
tfidf_train_feature = standard_tfidf_feature(tfidf_train_feature)
tfidf_dev_feature = standard_tfidf_feature(tfidf_dev_feature)
tfidf_test_feature = standard_tfidf_feature(tfidf_test_feature)
# node2vec
node2vec_train_feature = standard_node2vec_feature(node2vec_train_feature)
node2vec_dev_feature = standard_node2vec_feature(node2vec_dev_feature)
node2vec_test_feature = standard_node2vec_feature(node2vec_test_feature)

In [24]:
from keras.layers.core import Dense
from keras.layers.core import Activation
from keras.layers import concatenate
from keras.layers import Input
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.models import Model
from keras.losses import categorical_crossentropy
from keras import regularizers
from keras.utils.np_utils import to_categorical
# It is an original version of MENET model which just have 39% accuracy with the development dataset.

# # node2vec part
# x = Input(shape = (300,))
# y = Dense(150)(x)
# node2vec = Model(inputs = x, outputs = y)


# # tf-idf part
# x = Input(shape = (315,))
# y = Dense(150)(x)
# tfidf = Model(inputs = x, outputs = y)


# # glove 300 part
# x = Input(shape = (300,))
# y = Dense(30)(x)
# glove = Model(inputs = x, outputs = y)



# # MENET
# combined = concatenate([node2vec.output, tfidf.output, glove.output])
# relu = Dense(330, activation = "relu")(combined)
# result = Dense(4, activation = "softmax", kernel_regularizer = regularizers.l2(0.1))(relu)
# model = Model(inputs = [node2vec.input, tfidf.input, glove.input], outputs = result)


# opt = SGD(lr=1e-4, decay=1e-4 / 200)
# model.compile(loss = categorical_crossentropy,optimizer=opt,metrics=['accuracy'])
# cb = EarlyStopping(monitor='val_loss', patience=4,verbose=1)
# model.fit([node2vec_train_feature, tfidf_train_feature, glove_train_feature], to_categorical(standard_glove_train_label, num_classes = 4),\
#           validation_data=([node2vec_dev_feature, tfidf_dev_feature, glove_dev_feature], to_categorical(standard_glove_dev_label, num_classes = 4)),\
#           callbacks = [cb], epochs = 200)

In [23]:
# node2vec part
x = Input(shape = (300,))
y = Dense(150)(x)
node2vec = Model(inputs = x, outputs = y)

# tf-idf part
x = Input(shape = (315,))
y = Dense(150)(x)
tfidf = Model(inputs = x, outputs = y)

# MENET
combined = concatenate([node2vec.output, tfidf.output])
relu = Dense(300, activation = "relu")(combined)
result = Dense(4, activation = "softmax", kernel_regularizer = regularizers.l2(0.1))(relu)
i_model = Model(inputs = [node2vec.input, tfidf.input], outputs = result)


opt = Adam(lr=1e-4, decay=1e-4 / 200)
i_model.compile(loss = categorical_crossentropy,optimizer=opt,metrics=['accuracy'])
cb = EarlyStopping(monitor='val_loss', patience=4,verbose=1)
i_model.fit([node2vec_train_feature, f_tfidf_train_feature], to_categorical(standard_node2vec_train_label, num_classes = 4),\
          validation_data=([node2vec_dev_feature, f_tfidf_dev_feature], to_categorical(standard_node2vec_dev_label, num_classes = 4)),\
          callbacks = [cb], epochs = 200)

Train on 3400 samples, validate on 300 samples
Epoch 1/200
3400/3400 [==============================] - 0s 56us/step - loss: 1.9508 - accuracy: 0.4806 - val_loss: 1.9668 - val_accuracy: 0.3867
Epoch 2/200
3400/3400 [==============================] - 0s 32us/step - loss: 1.6281 - accuracy: 0.5988 - val_loss: 1.8441 - val_accuracy: 0.3967
Epoch 3/200
3400/3400 [==============================] - 0s 32us/step - loss: 1.4245 - accuracy: 0.6397 - val_loss: 1.7473 - val_accuracy: 0.4533
Epoch 4/200
3400/3400 [==============================] - 0s 32us/step - loss: 1.2771 - accuracy: 0.6844 - val_loss: 1.6685 - val_accuracy: 0.4567
Epoch 5/200
3400/3400 [==============================] - 0s 32us/step - loss: 1.1663 - accuracy: 0.7024 - val_loss: 1.6072 - val_accuracy: 0.4500
Epoch 6/200
3400/3400 [==============================] - 0s 31us/step - loss: 1.0801 - accuracy: 0.7138 - val_loss: 1.5508 - val_accuracy: 0.4533
Epoch 7/200
3400/3400 [==============================] - 0s 31us/step - loss:

In [12]:
# Multinomial naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(tfidf_train_feature, tfidf_train_label)
predict = clf.predict(tfidf_dev_feature)
print(accuracy(predict, tfidf_dev_label)) 
opredict = clf.predict(tfidf_train_feature)
print(accuracy(opredict, tfidf_train_label)) 

0.46352941176470586
0.49054897417691246


In [613]:
def out_file(filename, data):
    out_f = open("predict_f.csv", 'w', newline='')
    writer = csv.writer(out_f)
    head = ["id", "region"]
    writer.writerow(head)
    with open(filename) as f:
        reader = csv.reader(f)
        header = next(reader)
        user = []
        count = -1
        norm_count = 1
        for i in reader:
            if(i[1] not in user):
                count += 1
                user.append(i[1])
                writer.writerow((norm_count, data[count]))
            else:
                writer.writerow((norm_count, data[count]))
            norm_count += 1

In [664]:
result = get_result(model.predict([node2vec_test_feature, f_tfidf_test_feature]))

r_dict = {0: "MIDWEST", 1: "NORTHEAST", 2: "SOUTH", 3 : "WEST"}
text_result = []
for r in result:
    text_result.append(r_dict[r])
out_file('D:/comp90049/project3/data/test_full.csv', text_result)

In [1]:
from collections import Counter
def accuracy(predict, gold):
    acc = 0
    for pos in range(len(predict)):
        if(predict[pos] ==  gold[pos]):
            acc += 1
    accuracy = acc / len(predict)
    return accuracy
def zero_r_prediction(label):
    label_counter = Counter(label)
    zero_r_prediction = []
    majority_class = str(label_counter.most_common(1)[0][0])
    for i in range(len(label)):
        zero_r_prediction.append(majority_class)
    return zero_r_prediction


In [5]:
zero_r = zero_r_prediction(glove_train_label)
print(accuracy(zero_r, glove_train_label))

0.39300422287828396


In [6]:
zero_r = zero_r_prediction(glove_dev_label)
print(accuracy(zero_r, glove_dev_label))

0.37429193899782137
